In [17]:
from simple_idml import idml
from lxml import etree
import xml.etree.ElementTree as ET

In [18]:
idml = idml.IDMLPackage("./Manuscript/two.idml")
idml.extractall("./documents/")

In [23]:
'''
    GET PAGES BY SPREADS
'''
for spread in idml.spreads:
    tree = ET.parse("./Manuscript/documents/" + spread)
    root = tree.getroot()
    for item in root.iter():
        print(item.tag)

{http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging}Spread
Spread
FlattenerPreference
Properties
RasterVectorBalance
Page
Properties
PageColor
Descriptor
ListItem
ListItem
ListItem
ListItem
ListItem
ListItem
MarginPreference
GridDataInformation
Properties
AppliedFont
TextFrame
Properties
PathGeometry
GeometryPathType
PathPointArray
PathPointType
PathPointType
PathPointType
PathPointType
ObjectExportOption
Properties
AltMetadataProperty
ActualMetadataProperty
TextFramePreference
Properties
InsetSpacing
ListItem
ListItem
ListItem
ListItem
TextWrapPreference
Properties
TextWrapOffset
TextFrame
Properties
PathGeometry
GeometryPathType
PathPointArray
PathPointType
PathPointType
PathPointType
PathPointType
ObjectExportOption
Properties
AltMetadataProperty
ActualMetadataProperty
TextFramePreference
Properties
InsetSpacing
ListItem
ListItem
ListItem
ListItem
TextWrapPreference
Properties
TextWrapOffset


In [49]:
'''
    GET PAGE SIZE (width - 498, height - 708) IN POINTS
    -------------------------------------------------------
    Measurement units in IDML exported from In
    Design are always points (defined as 72 units per 
    inch).
'''
for page in idml.pages:
    for items in page.spread.pages:
        print(items)

In [50]:
'''
    GET PAGES WITHIN THE SPREAD
'''
for spread in idml.pages:
    print(spread.spread.pages)

[<simple_idml.components.Page object at 0x106929ba8>, <simple_idml.components.Page object at 0x106d20d30>]
[<simple_idml.components.Page object at 0x106929ba8>, <simple_idml.components.Page object at 0x106d20d30>]
[<simple_idml.components.Page object at 0x107346240>, <simple_idml.components.Page object at 0x107134748>]
[<simple_idml.components.Page object at 0x107346240>, <simple_idml.components.Page object at 0x107134748>]
[<simple_idml.components.Page object at 0x107134d68>, <simple_idml.components.Page object at 0x107134710>]
[<simple_idml.components.Page object at 0x107134d68>, <simple_idml.components.Page object at 0x107134710>]
[<simple_idml.components.Page object at 0x107134470>, <simple_idml.components.Page object at 0x107134780>]
[<simple_idml.components.Page object at 0x107134470>, <simple_idml.components.Page object at 0x107134780>]


In [51]:
'''
    Extract the xml files from provided indesign file
'''
idml.extractall("./documents/")

In [73]:
idml.graphic

<Graphic object Resources/Graphic.xml at 0x10702f3c8>

In [6]:
tree = ET.parse('../idml2html/' + idml.spreads[0])
print(ET.tostring(tree.getroot(), encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:Spread xmlns:ns0="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
	<Spread AllowPageShuffle="true" BindingLocation="0" FlattenerOverride="Default" ItemTransform="1 0 0 1 0 0" PageCount="1" PageTransitionDirection="NotApplicable" PageTransitionDuration="Medium" PageTransitionType="None" Self="uc8" ShowMasterItems="true">
		<FlattenerPreference ClipComplexRegions="false" ConvertAllStrokesToOutlines="false" ConvertAllTextToOutlines="false" GradientAndMeshResolution="150" LineArtAndTextResolution="300">
			<Properties>
				<RasterVectorBalance type="double">50</RasterVectorBalance>
			</Properties>
		</FlattenerPreference>
		<Page AppliedAlternateLayout="uf0" AppliedMaster="uf1" AppliedTrapPreset="TrapPreset/$ID/kDefaultTrapStyleName" GeometricBounds="0 0 708.6614173237 498.8976377953" GridStartingPoint="TopOutside" ItemTransform="1 0 0 1 0 -354.33070866185" LayoutRule="UseMaster" MasterPageTransform="1 0 0 1 0 0" Name="1"

In [7]:
tree = ET.parse('../idml2html/' + idml.spreads[1])
print(ET.tostring(tree.getroot(), encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:Spread xmlns:ns0="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
	<Spread AllowPageShuffle="true" BindingLocation="1" FlattenerOverride="Default" ItemTransform="1 0 0 1 0 888.6614173237" PageCount="2" PageTransitionDirection="NotApplicable" PageTransitionDuration="Medium" PageTransitionType="None" Self="uce" ShowMasterItems="true">
		<FlattenerPreference ClipComplexRegions="false" ConvertAllStrokesToOutlines="false" ConvertAllTextToOutlines="false" GradientAndMeshResolution="150" LineArtAndTextResolution="300">
			<Properties>
				<RasterVectorBalance type="double">50</RasterVectorBalance>
			</Properties>
		</FlattenerPreference>
		<Page AppliedAlternateLayout="uf0" AppliedMaster="uf1" AppliedTrapPreset="TrapPreset/$ID/kDefaultTrapStyleName" GeometricBounds="0 0 708.6614173237 498.8976377953" GridStartingPoint="TopOutside" ItemTransform="1 0 0 1 -498.8976377953 -354.33070866185" LayoutRule="UseMaster" MasterPageTrans

In [8]:
# for spread in idml.spreads:
#     tree = ET.parse('../idml2html/' + spread)
#     root = tree.getroot()
#     for page in root.iter("Page"):
#         print(page.attrib)

tree = ET.parse('../idml2html/' + idml.stories[0])
print(ET.tostring(tree.getroot(), encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:Story xmlns:ns0="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
	<Story AppliedNamedGrid="n" AppliedTOCStyle="n" IsEndnoteStory="false" Self="u2ee" StoryTitle="$ID/" TrackChanges="false" UserText="true">
		<StoryPreference FrameType="TextFrameType" OpticalMarginAlignment="false" OpticalMarginSize="12" StoryDirection="LeftToRightDirection" StoryOrientation="Horizontal" />
		<InCopyExportOption IncludeAllResources="false" IncludeGraphicProxies="true" />
		<ParagraphStyleRange AppliedParagraphStyle="ParagraphStyle/$ID/NormalParagraphStyle">
			<CharacterStyleRange AppliedCharacterStyle="CharacterStyle/$ID/[No character style]">
				<Content>Building high quality training datasets is one of the most difficult challenges of machine learning solutions in the real world. Disciplines like deep learning have helped us to build more accurate models but, to do so, they require vastly larger volumes of training data. Now, saying 

In [9]:
idml.stories

['Stories/Story_u2ee.xml', 'Stories/Story_u1b1.xml']

In [10]:
tree = ET.parse('../idml2html/' + idml.stories[0])
print(ET.tostring(tree.getroot(), encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:Story xmlns:ns0="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
	<Story AppliedNamedGrid="n" AppliedTOCStyle="n" IsEndnoteStory="false" Self="u2ee" StoryTitle="$ID/" TrackChanges="false" UserText="true">
		<StoryPreference FrameType="TextFrameType" OpticalMarginAlignment="false" OpticalMarginSize="12" StoryDirection="LeftToRightDirection" StoryOrientation="Horizontal" />
		<InCopyExportOption IncludeAllResources="false" IncludeGraphicProxies="true" />
		<ParagraphStyleRange AppliedParagraphStyle="ParagraphStyle/$ID/NormalParagraphStyle">
			<CharacterStyleRange AppliedCharacterStyle="CharacterStyle/$ID/[No character style]">
				<Content>Building high quality training datasets is one of the most difficult challenges of machine learning solutions in the real world. Disciplines like deep learning have helped us to build more accurate models but, to do so, they require vastly larger volumes of training data. Now, saying 

In [123]:
tree = ET.parse('../idml2html/' + idml.graphic.name)
for i in tree.getroot().iter("Color"):
    print(i.attrib)

{'Self': 'Color/Black', 'Model': 'Process', 'Space': 'CMYK', 'ColorValue': '0 0 0 100', 'ColorOverride': 'Specialblack', 'AlternateSpace': 'NoAlternateColor', 'AlternateColorValue': '', 'Name': 'Black', 'ColorEditable': 'false', 'ColorRemovable': 'false', 'Visible': 'true', 'SwatchCreatorID': '7937', 'SwatchColorGroupReference': 'u18ColorGroupSwatch3'}
{'Self': 'Color/C=0 M=0 Y=100 K=0', 'Model': 'Process', 'Space': 'CMYK', 'ColorValue': '0 0 100 0', 'ColorOverride': 'Normal', 'AlternateSpace': 'NoAlternateColor', 'AlternateColorValue': '', 'Name': 'C=0 M=0 Y=100 K=0', 'ColorEditable': 'true', 'ColorRemovable': 'true', 'Visible': 'true', 'SwatchCreatorID': '7937', 'SwatchColorGroupReference': 'u18ColorGroupSwatch6'}
{'Self': 'Color/C=0 M=100 Y=0 K=0', 'Model': 'Process', 'Space': 'CMYK', 'ColorValue': '0 100 0 0', 'ColorOverride': 'Normal', 'AlternateSpace': 'NoAlternateColor', 'AlternateColorValue': '', 'Name': 'C=0 M=100 Y=0 K=0', 'ColorEditable': 'true', 'ColorRemovable': 'true', 'V

In [11]:
def get_alignment(args): 
    switcher = { 
        "CenterAlign": "text-align: center", 
        "RightAlign": "text-align: right", 
        "LeftAlign": "text-align: left", 
    } 
  
    return switcher.get(args, switcher["LeftAlign"]) 

def get_font_weight(args):
    switcher = { 
        "Bold": "font-weight: bold", 
        "Italics": "font-weight: italics", 
        "Normal": "font-weight: normal", 
    }
  
    return switcher.get(args, switcher["Normal"]) 

def get_font_size(args):
    return "font-size: " + args + "pt" if args else "font-size: 8pt"


# Parent styles

finalChild = ""

for story in idml.stories:
    tree = ET.parse('../idml2html/' + story)
    root = tree.getroot()
    strContent = ''
    
    paragraphStyleRanges = root.iter("ParagraphStyleRange")
    
    for paragraphStyleRange in paragraphStyleRanges:
        parentStyle = ""
        alignment = paragraphStyleRange.attrib.get("Justification")
        alignment = get_alignment(alignment)
        parentStyle += "<p style='" + alignment + "'>"
        
        # Parent styles
        characterStyleRanges = paragraphStyleRange.iter('CharacterStyleRange')
        childStyle = ""
        for characterStyleRange in characterStyleRanges:
            fontStyle = characterStyleRange.attrib.get("FontStyle")
            fontStyle = get_font_weight(fontStyle)
            
            fontSize = characterStyleRange.attrib.get("PointSize")
            fontSize = get_font_size(fontSize)
            
            childrens = [i for i in characterStyleRange.iter()]
            
            for child in childrens:
                if child.tag == "Content":
                    childStyle += "<span style='" + fontStyle + ";"
                    childStyle += fontSize + ";" if fontSize else ""
                    childStyle += "'>" + child.text if child.text else "'>" 
                    childStyle += "</span>"
                
                if child.tag == "Br":
                    childStyle += "<br />"
                    
        finalChild += childStyle
                                 
strContent += parentStyle + finalChild + "</p>"

print(strContent)

<p style='text-align: left'><span style='font-weight: normal;font-size: 8pt;'>Building high quality training datasets is one of the most difficult challenges of machine learning solutions in the real world. Disciplines like deep learning have helped us to build more accurate models but, to do so, they require vastly larger volumes of training data. Now, saying that effective machine learning requires a lot of training data is like saying that “you need a lot of money to be rich”. It’s true, but it doesn’t make it less painful to get there. In many of the machine learning projects we work on at Invector Labs, our customers spend significant more time collecting and labeling training dataset than building machine learning models. Last year, we came across a small project created by artificial intelligence(AI) researchers from Stanford University that provides a programming model for the creation of training datasets. Ever since, Snorkel has become a regular component of our machine learn

In [12]:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<idPkg:MasterSpread xmlns:idPkg="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
        <MasterSpread Self="ub2" Name="A-Master" NamePrefix="A" BaseName="Master" ShowMasterItems="true" PageCount="2" OverriddenPageItemProps="" PrimaryTextFrame="n" ItemTransform="1 0 0 1 0 0">
                <Properties>
                        <PageColor type="enumeration">UseMasterColor</PageColor>
                </Properties>
                <Page Self="ub7" TabOrder="" AppliedMaster="n" OverrideList="" MasterPageTransform="1 0 0 1 0 0" Name="A" AppliedTrapPreset="TrapPreset/$ID/kDefaultTrapStyleName" GeometricBounds="-18 -35.99999999999999 594 360"ItemTransform="1 0 0 1 -360 -288" AppliedAlternateLayout="n" LayoutRule="Off" SnapshotBlendingMode="IgnoreLayoutSnapshots" OptionalPage="false" GridStartingPoint="TopOutside" UseMasterGrid="true">
                        <Properties>
                                <PageColor type="enumeration">UseMasterColor</PageColor>
                        </Properties>
                        <MarginPreference ColumnCount="1" ColumnGutter="12" Top="61.199999999999996" Bottom="72" Left="54" Right="54" ColumnDirection="Horizontal" ColumnsPositions="0 288" />
                        <GridDataInformation FontStyle="Regular" PointSize="12" CharacterAki="0" LineAki="9" HorizontalScale="100" VerticalScale="100" LineAlignment="LeftOrTopLineJustify" GridAlignment="AlignEmCenter" CharacterAlignment="AlignEmCenter">
                                <Properties>
                                        <AppliedFont type="string">Minion Pro</AppliedFont>
                                </Properties>
                        </GridDataInformation>
                </Page>
                <Page Self="ub8" TabOrder="" AppliedMaster="n" OverrideList="" MasterPageTransform="1 0 0 1 0 0" Name="A" AppliedTrapPreset="TrapPreset/$ID/kDefaultTrapStyleName" GeometricBounds="-18 0 594 396" ItemTransform="10 0 1 0 -288" AppliedAlternateLayout="n" LayoutRule="Off" SnapshotBlendingMode="IgnoreLayoutSnapshots" OptionalPage="false" GridStartingPoint="TopOutside" UseMasterGrid="true">
                        <Properties>
                                <PageColor type="enumeration">UseMasterColor</PageColor>
                        </Properties>
                        <MarginPreference ColumnCount="1" ColumnGutter="12" Top="61.199999999999996" Bottom="72" Left="54" Right="54" ColumnDirection="Horizontal" ColumnsPositions="0 288" />
                        <GridDataInformation FontStyle="Regular" PointSize="12" CharacterAki="0" LineAki="9" HorizontalScale="100" VerticalScale="100" LineAlignment="LeftOrTopLineJustify" GridAlignment="AlignEmCenter" CharacterAlignment="AlignEmCenter">
                                <Properties>
                                        <AppliedFont type="string">Minion Pro</AppliedFont>
                                </Properties>
                        </GridDataInformation>
                </Page>
                <TextFrame Self="u1a9e" ParentStory="u1aa1" PreviousTextFrame="n" NextTextFrame="n" ContentType="TextType" OverriddenPageItemProps="" AllowOverrides="true" Visible="true" Name="$ID/" HorizontalLayoutConstraints="FlexibleDimension FixedDimension FlexibleDimension" VerticalLayoutConstraints="FlexibleDimension FixedDimension FlexibleDimension" GradientFillStart="0 0" GradientFillLength="0" GradientFillAngle="0" GradientStrokeStart="0 0" GradientStrokeLength="0" GradientStrokeAngle="0" ItemLayer="u110" Locked="false" LocalDisplaySetting="Default" GradientFillHiliteLength="0" GradientFillHiliteAngle="0" GradientStrokeHiliteLength="0" GradientStrokeHiliteAngle="0" AppliedObjectStyle="ObjectStyle/$ID/[Normal Text Frame]" ItemTransform="1 0 0 1 -220.66193847656245 263.5199999999984" ParentInterfaceChangeCount="" TargetInterfaceChangeCount="" LastUpdatedInterfaceChangeCount="">
                        <Properties>
                                <PathGeometry>
                                        <GeometryPathType PathOpen="false">
                                                <PathPointArray>
                                                        <PathPointType Anchor="15.183132934570267 -11.51999999999839" LeftDirection="15.183132934570267 -11.51999999999839" RightDirection="15.183132934570267 -11.51999999999839" />
                                                        <PathPointType Anchor="15.183132934570267 -4.518001098631203" LeftDirection="15.183132934570267 -4.518001098631203" RightDirection="15.183132934570267 -4.518001098631203" />
                                                        <PathPointType Anchor="30.140744018554642 -4.518001098631203" LeftDirection="30.140744018554642 -4.518001098631203" RightDirection="30.140744018554642 -4.518001098631203" />
                                                        <PathPointType Anchor="30.140744018554642 -11.51999999999839" LeftDirection="30.140744018554642 -11.51999999999839" RightDirection="30.140744018554642 -11.51999999999839" />
                                                </PathPointArray>
                                        </GeometryPathType>
                                </PathGeometry>
                        </Properties>
                        <ObjectExportOption AltTextSourceType="SourceXMLStructure" ActualTextSourceType="SourceXMLStructure" CustomAltText="$ID/" CustomActualText="$ID/" ApplyTagType="TagFromStructure" ImageConversionType="JPEG" ImageExportResolution="Ppi300" GIFOptionsPalette="AdaptivePalette" GIFOptionsInterlaced="true" JPEGOptionsQuality="High" JPEGOptionsFormat="BaselineEncoding" ImageAlignment="AlignLeft" ImageSpaceBefore="0" ImageSpaceAfter="0" UseImagePageBreak="false" ImagePageBreak="PageBreakBefore" CustomImageAlignment="false" SpaceUnit="CssPixel" CustomLayout="false" CustomLayoutType="AlignmentAndSpacing" EpubType="$ID/" SizeType="DefaultSize" CustomSize="$ID/" PreserveAppearanceFromLayout="PreserveAppearanceDefault">
                                <Properties>
                                        <AltMetadataProperty NamespacePrefix="$ID/" PropertyPath="$ID/" />
                                        <ActualMetadataProperty NamespacePrefix="$ID/" PropertyPath="$ID/" />
                                </Properties>
                        </ObjectExportOption>
                        <TextFramePreference FootnotesEnableOverrides="false" FootnotesSpanAcrossColumns="false" FootnotesMinimumSpacing="12" FootnotesSpaceBetween="6" TextColumnCount="1" TextColumnFixedWidth="14.957611083984375" VerticalJustification="CenterAlign" TextColumnMaxWidth="0" AutoSizingType="Off" AutoSizingReferencePoint="CenterPoint" UseMinimumHeightForAutoSizing="false" MinimumHeightForAutoSizing="0" UseMinimumWidthForAutoSizing="false" MinimumWidthForAutoSizing="0" UseNoLineBreaksForAutoSizing="false">
                                <Properties>
                                        <InsetSpacing type="list">
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                        </InsetSpacing>
                                </Properties>
                        </TextFramePreference>
                        <TextFrameFootnoteOptionsObject EnableOverrides="false" SpanFootnotesAcross="false" MinimumSpacingOption="12" SpaceBetweenFootnotes="6" />
                        <TextWrapPreference Inverse="false" ApplyToMasterPageOnly="false" TextWrapSide="BothSides" TextWrapMode="None">
                                <Properties>
                                        <TextWrapOffset Top="0" Left="0" Bottom="0" Right="0" />
                                </Properties>
                        </TextWrapPreference>
                </TextFrame>
                <TextFrame Self="u1ab6" ParentStory="u1ab9" PreviousTextFrame="n" NextTextFrame="n" ContentType="TextType" OverriddenPageItemProps="" AllowOverrides="true" Visible="true" Name="$ID/" HorizontalLayoutConstraints="FlexibleDimension FixedDimension FlexibleDimension" VerticalLayoutConstraints="FlexibleDimension FixedDimension FlexibleDimension" GradientFillStart="0 0" GradientFillLength="0" GradientFillAngle="0" GradientStrokeStart="0 0" GradientStrokeLength="0" GradientStrokeAngle="0" ItemLayer="u110" Locked="false" LocalDisplaySetting="Default" GradientFillHiliteLength="0" GradientFillHiliteAngle="0" GradientStrokeHiliteLength="0" GradientStrokeHiliteAngle="0" AppliedObjectStyle="ObjectStyle/$ID/[Normal Text Frame]" ItemTransform="1 0 0 1 196.28119445800752 263.52000000000055" ParentInterfaceChangeCount="" TargetInterfaceChangeCount="" LastUpdatedInterfaceChangeCount="">
                        <Properties>
                                <PathGeometry>
                                        <GeometryPathType PathOpen="false">
                                                <PathPointArray>
                                                        <PathPointType Anchor="-5.75999999999965 -11.52000000000055" LeftDirection="-5.75999999999965 -11.52000000000055" RightDirection="-5.75999999999965 -11.52000000000055" />
                                                        <PathPointType Anchor="-5.75999999999965 -4.518001098633363" LeftDirection="-5.75999999999965 -4.518001098633363" RightDirection="-5.75999999999965 -4.518001098633363" />
                                                        <PathPointType Anchor="9.197611083984608 -4.518001098633363" LeftDirection="9.197611083984608 -4.518001098633363" RightDirection="9.197611083984608 -4.518001098633363" />
                                                        <PathPointType Anchor="9.197611083984608 -11.52000000000055" LeftDirection="9.197611083984608 -11.52000000000055" RightDirection="9.197611083984608 -11.52000000000055" />
                                                </PathPointArray>
                                        </GeometryPathType>
                                </PathGeometry>
                        </Properties>
                        <ObjectExportOption AltTextSourceType="SourceXMLStructure" ActualTextSourceType="SourceXMLStructure" CustomAltText="$ID/" CustomActualText="$ID/" ApplyTagType="TagFromStructure" ImageConversionType="JPEG" ImageExportResolution="Ppi300" GIFOptionsPalette="AdaptivePalette" GIFOptionsInterlaced="true" JPEGOptionsQuality="High" JPEGOptionsFormat="BaselineEncoding" ImageAlignment="AlignLeft" ImageSpaceBefore="0" ImageSpaceAfter="0" UseImagePageBreak="false" ImagePageBreak="PageBreakBefore" CustomImageAlignment="false" SpaceUnit="CssPixel" CustomLayout="false" CustomLayoutType="AlignmentAndSpacing" EpubType="$ID/" SizeType="DefaultSize" CustomSize="$ID/" PreserveAppearanceFromLayout="PreserveAppearanceDefault">
                                <Properties>
                                        <AltMetadataProperty NamespacePrefix="$ID/" PropertyPath="$ID/" />
                                        <ActualMetadataProperty NamespacePrefix="$ID/" PropertyPath="$ID/" />
                                </Properties>
                        </ObjectExportOption>
                        <TextFramePreference FootnotesEnableOverrides="false" FootnotesSpanAcrossColumns="false" FootnotesMinimumSpacing="12" FootnotesSpaceBetween="6" TextColumnCount="1" TextColumnFixedWidth="14.957611083984375" VerticalJustification="CenterAlign" TextColumnMaxWidth="0" AutoSizingType="Off" AutoSizingReferencePoint="CenterPoint" UseMinimumHeightForAutoSizing="false" MinimumHeightForAutoSizing="0" UseMinimumWidthForAutoSizing="false" MinimumWidthForAutoSizing="0" UseNoLineBreaksForAutoSizing="false">
                                <Properties>
                                        <InsetSpacing type="list">
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                                <ListItem type="unit">0</ListItem>
                                        </InsetSpacing>
                                </Properties>
                        </TextFramePreference>
                        <TextFrameFootnoteOptionsObject EnableOverrides="false" SpanFootnotesAcross="false" MinimumSpacingOption="12" SpaceBetweenFootnotes="6" />
                        <TextWrapPreference Inverse="false" ApplyToMasterPageOnly="false" TextWrapSide="BothSides" TextWrapMode="None">
                                <Properties>
                                        <TextWrapOffset Top="0" Left="0" Bottom="0" Right="0" />
                                </Properties>
                        </TextWrapPreference>
                </TextFrame>
        </MasterSpread>
</idPkg:MasterSpread>

SyntaxError: invalid syntax (<ipython-input-12-38d1c85262c8>, line 1)